In [1]:
from utils.config import Config
from model.fasterrcnn import FasterRCNNTrainer, FasterRCNN
import tensorflow as tf
from utils.data import Dataset

In [2]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [3]:
%matplotlib notebook

In [4]:
config = Config()
config._parse({})

======user config========
{'max_size': 1000,
 'min_size': 600,
 'voc_data_dir': 'D:/DataClinic/simple-faster-rcnn-pytorch/VOCdevkit/VOC2007/'}
==========end============


In [5]:
dataset = Dataset(config)

In [7]:
frcnn = FasterRCNN(21, (7,7))
model = FasterRCNNTrainer(frcnn)
optimizer = tf.keras.optimizers.SGD(learning_rate=3e-4)

In [8]:
img, bboxes, labels, scale = dataset[1]

In [9]:
img_size = img.shape[1:3]
feature_map = model.faster_rcnn.extractor(img)
rpn_locs, rpn_scores, rois, anchor = model.faster_rcnn.rpn(feature_map, img_size, scale)

In [10]:
print(feature_map.shape, 'max=', tf.reduce_max(feature_map).numpy(), 'min=', tf.reduce_min(feature_map).numpy())

(1, 38, 57, 512) max= 0.029995251 min= 0.0


In [11]:
print(rpn_locs.shape, 'max=', tf.reduce_max(rpn_locs).numpy(), 'min=', tf.reduce_min(feature_map).numpy())
print(rpn_scores.shape,'max=', tf.reduce_max(rpn_scores).numpy(), 'min=', tf.reduce_min(rpn_scores).numpy())

(1, 19494, 4) max= 0.013715236 min= 0.0
(1, 19494, 2) max= 0.017642885 min= -0.010967214


In [12]:
print(rois.shape,'max=', tf.reduce_max(rois).numpy(), 'min=', tf.reduce_min(rois).numpy())
print(roi_indices.shape,'max=', tf.reduce_max(roi_indices).numpy(), 'min=', tf.reduce_min(roi_indices).numpy())
print(anchor.shape,'max=', tf.reduce_max(anchor).numpy(), 'min=', tf.reduce_min(anchor).numpy())

(2000, 4) max= 800.0 min= 0.0
(2000,) max= 0 min= 0
(17100, 4) max= 1154.0387 min= -354.03867


In [16]:
from utils.anchor import AnchorTargetCreator

In [17]:
anchor_target_creator = AnchorTargetCreator(n_sample=256, pos_iou_thresh=0.7, neg_iou_thresh=0.3, pos_ratio=0.5)

In [19]:
img_size = img.shape[:2]

In [21]:
gt_rpn_loc, gt_rpn_label = anchor_target_creator(bboxes, anchor, img_size)

In [22]:
print(gt_rpn_loc.shape, 'max=', tf.reduce_max(gt_rpn_loc).numpy(), 'min=', tf.reduce_min(gt_rpn_loc).numpy())
print(gt_rpn_label.shape,'max=', tf.reduce_max(gt_rpn_label).numpy(), 'min=', tf.reduce_min(gt_rpn_label).numpy())

(17100, 4) max= 4.110058 min= -3.9244435
(17100,) max= 1 min= -1


In [27]:
from utils.anchor import ProposalTargetCreator

In [29]:
proposal_target_creator = ProposalTargetCreator(n_sample=128, pos_ratio=0.25, pos_iou_thresh=0.5, 
                                                neg_iou_thresh_hi=0.5, neg_iou_thresh_lo=0.0)

In [30]:
sample_roi, gt_roi_loc, gt_roi_label, keep_index = proposal_target_creator(rois, bboxes, labels)

In [31]:
print(sample_roi.shape, 'max=', tf.reduce_max(sample_roi).numpy(), 'min=', tf.reduce_min(sample_roi).numpy())
print(gt_roi_loc.shape,'max=', tf.reduce_max(gt_roi_loc).numpy(), 'min=', tf.reduce_min(gt_roi_loc).numpy())
print(gt_roi_label.shape, 'max=', tf.reduce_max(gt_roi_label).numpy(), 'min=', tf.reduce_min(gt_roi_label).numpy())
print(keep_index.shape,'max=', tf.reduce_max(keep_index).numpy(), 'min=', tf.reduce_min(keep_index).numpy())

(128, 4) max= 800.0 min= 0.0
(128, 4) max= 6.2737284 min= -4.2205048
(128,) max= 15 min= 0
(128,) max= 2001 min= 0


In [33]:
keep_index[:10]

array([1144, 1386,    2, 1275, 1292, 1168, 1825,  691, 1889, 2000],
      dtype=int64)

In [36]:
from model.roi import RoIHead

In [60]:
roi_head = RoIHead(n_class=21, pool_size=(7, 7))

In [61]:
roi_cls_loc, roi_score = roi_head(feature_map, sample_roi, img_size)

In [62]:
print(roi_cls_loc.shape, 'max=', tf.reduce_max(roi_cls_loc).numpy(), 'min=', tf.reduce_min(roi_cls_loc).numpy())
print(roi_score.shape,'max=', tf.reduce_max(roi_score).numpy(), 'min=', tf.reduce_min(roi_score).numpy())

(128, 84) max= 0.050996922 min= -0.04852302
(128, 21) max= 0.5078401 min= 0.49183586
